In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
from skimage import io, color, exposure, transform
from time import time
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import glob
import h5py

/home/citlalli/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# --------------------- Define functions -------------------------------
def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)
    
    # Re-scale image
    img = transform.resize(img, (IMAGE_DIMS[1], IMAGE_DIMS[0]))

    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])


## Read Data set

In [3]:
#------- Read training images---------------------------------------------------
NUM_CLASSES = 43 # 43 for GTSRB, 164 for European data set
IMAGE_DIMS = (48,48,3)
file_name = 'X_train_german.h5' # File name for saving your training images

try:
    with  h5py.File(file_name) as hf: 
        X, Y = hf['imgs'][:], hf['labels'][:]
    print("Loaded images from {}".format(file_name))
    
except (IOError,OSError, KeyError):  
    print("Error in reading {}. Processing all images...".format(file_name))
    root_dir = '../../Datasets/Traffic_signs/GTSRB/Final_Training/Images' # changed to your needs
    imgs = []
    labels = []

    # Read all image paths with extension ppm
    all_img_paths = sorted(glob.glob(os.path.join(root_dir, '*/*.ppm')) )
    np.random.seed(42)
    np.random.shuffle(all_img_paths)
    
    # Read images and preprocess them
    print("[INFO] loading images...")
    for img_path in all_img_paths:
        try:
            img = preprocess_img(io.imread(img_path))            
            label = get_class(img_path)            
            
            # Save images and labels in lists
            imgs.append(img)
            labels.append(label)

            if len(imgs)%1000 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
        except (IOError, OSError):
            print('missed', img_path)
            pass

    X = np.array(imgs, dtype='float32')
    Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    # Save the training dictionary
    with h5py.File(file_name,'w') as hf:
        hf.create_dataset('imgs', data=X)
        hf.create_dataset('labels', data=Y)




Loaded images from X_train_german.h5


In [4]:
#--------------- Read test images-------------------------------
file_name = 'X_test_german.h5' # File name for saving your testing images

try:
    with  h5py.File(file_name) as hf: 
        X_test, y_test = hf['imgs'][:], hf['labels'][:]
    print("Loaded images from {}".format(file_name))
    
except (IOError,OSError, KeyError):  
    print("Error in reading {}. Processing all images...".format(file_name))
    root_dir = '../../Datasets/Traffic_signs/GTSRB/Final_Test/Images2'
    X_test = []
    y_test = []

    # Read all image paths with extension ppm
    all_img_paths = sorted(glob.glob(os.path.join(root_dir, '*/*.ppm')) )
    np.random.seed(42)
    np.random.shuffle(all_img_paths)

    # Read images and preprocess them    
    for img_path in all_img_paths:
        try:
            img = preprocess_img(io.imread(img_path))            
            label = get_class(img_path)
            
            # Save images and labels in lists
            X_test.append(img)
            y_test.append(label)

            if len(X_test)%1000 == 0: print("Processed {}/{}".format(len(X_test), len(all_img_paths)))
        except (IOError, OSError):
            print('missed', img_path)
            pass

    X_test = np.array(X_test, dtype='float32')
    y_test = np.array(y_test, dtype='uint8')
    # Save testing dictionary
    with h5py.File(file_name,'w') as hf:
        hf.create_dataset('imgs', data=X_test)
        hf.create_dataset('labels', data=y_test)


Loaded images from X_test_german.h5


In [5]:
# ---------------------- Split dataset into train and validation -----------------------------
# partition the data into training and validation sets using 90% - 10% ratio
split = train_test_split(X, Y, test_size=0.1, random_state=42)
(X_train, X_val, y_train, y_val) = split

In [6]:
# --------------------- Normalize the data ----------------------------------
normalize = 0
# Subtract the mean image
if normalize:#
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image
    X_val -= mean_image

print('Train data shape: ', X_train.shape)
print('Train classes labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation classes labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test classes labels shape: ', y_test.shape)

Train data shape:  (35288, 48, 48, 3)
Train classes labels shape:  (35288, 43)
Validation data shape:  (3921, 48, 48, 3)
Validation classes labels shape:  (3921, 43)
Test data shape:  (12630, 48, 48, 3)
Test classes labels shape:  (12630,)


## CNN 8-layers model

In [7]:
# ----------------- Initialization of weights -------------------------------
from keras import backend as K
import numpy as np

def my_init(shape, name=None):
    value = np.random.random(shape)
    return K.variable(value, name=name)

In [8]:
# --------------- Define Keras model ---------------------------------- 
from model.TS_Net_proposal import TrafficSignsNet

model = TrafficSignsNet.build(48, 48,
    numClasses=43,
    finalAct="softmax")

model.summary() # Print model information

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
__________

In [9]:
#--------------------- Initialize model ------------------------
EPOCHS = 40
INIT_LR = 1e-3
BS = 128

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

def lr_schedule(epoch):
    return lr * (0.1 ** init(epoch/10))

In [10]:
#---------------------- Train model -----------------------
t1=time()
history = model.fit(X_train, y_train, batch_size=BS, 
                            nb_epoch=EPOCHS,
                            validation_data=(X_val, y_val),
                            shuffle = True
         )
t2 = time()

print ('Time taken to train the model', t2-t1)

/home/citlalli/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 35288 samples, validate on 3921 samples
Epoch 1/40
35288/35288 [==============================] - 27s 759us/step - loss: 0.9790 - acc: 0.7869 - val_loss: 0.2982 - val_acc: 0.9617
Epoch 2/40
35288/35288 [==============================] - 24s 672us/step - loss: 0.2669 - acc: 0.9772 - val_loss: 0.2066 - val_acc: 0.9913
Epoch 3/40
35288/35288 [==============================] - 24s 675us/step - loss: 0.2127 - acc: 0.9889 - val_loss: 0.1824 - val_acc: 0.9946
Epoch 4/40
35288/35288 [==============================] - 24s 676us/step - loss: 0.1932 - acc: 0.9922 - val_loss: 0.1691 - val_acc: 0.9967
Epoch 5/40
35288/35288 [==============================] - 24s 675us/step - loss: 0.1803 - acc: 0.9938 - val_loss: 0.1757 - val_acc: 0.9939
Epoch 6/40
35288/35288 [==============================] - 24s 674us/step - loss: 0.1766 - acc: 0.9933 - val_loss: 0.1697 - val_acc: 0.9941
Epoch 7/40
35288/35288 [==============================] - 24s 676us/step - loss: 0.1714 - acc: 0.9940 - val_loss: 0.1

## Test on the Test Dataset

In [8]:
t1=time()
y_proba = model.predict(X_test)
t2=time()
print  ('Took {} sec to predict {} images'.format(t2-t1, len(X_test)) )

pred_labels = []
pred_probas = []
for i in range(0,len(y_proba)):
    classId = y_proba[i].argmax()
    pred_labels.append(classId)
    proba = max(y_proba[i])
    pred_probas.append(proba)

pred_labels = np.array(pred_labels, dtype='uint8')
pred_probas = np.array(pred_probas)

acc = np.mean(pred_labels==y_test)
print("Test accuracy = {}".format(acc*100))

Took 2.7618215084075928 sec to predict 12630 images
Test accuracy = 98.5193982581156


In [60]:
model.save('GTSRB_weights/german_model_98.52.h5')

## Train with data augmentation

In [13]:
datagen = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)
datagen.fit(X_train)


In [14]:
nb_epoch = 50
history2=model.fit_generator(datagen.flow(X_train, y_train, batch_size=BS),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, y_val),
                            callbacks=[ReduceLROnPlateau('val_loss', factor=0.2, patience=20, verbose=1, mode='auto'), 
                                       ModelCheckpoint('GTSRB_weights/german_aug_test.h5',save_best_only=True)]
                           )


Epoch 1/50
  2/275 [..............................] - ETA: 26s - loss: 0.5981 - acc: 0.8789

/home/citlalli/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/citlalli/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=(array([[[..., callbacks=[<keras.ca..., steps_per_epoch=275)`
  import sys


275/275 [==============================] - 18s 65ms/step - loss: 0.2722 - acc: 0.9717 - val_loss: 0.1871 - val_acc: 0.9969
Epoch 2/50
275/275 [==============================] - 18s 66ms/step - loss: 0.2130 - acc: 0.9893 - val_loss: 0.1818 - val_acc: 0.9964
Epoch 3/50
275/275 [==============================] - 18s 67ms/step - loss: 0.1958 - acc: 0.9914 - val_loss: 0.1703 - val_acc: 0.9977
Epoch 4/50
275/275 [==============================] - 18s 67ms/step - loss: 0.1873 - acc: 0.9918 - val_loss: 0.1669 - val_acc: 0.9969
Epoch 5/50
275/275 [==============================] - 18s 67ms/step - loss: 0.1789 - acc: 0.9926 - val_loss: 0.1561 - val_acc: 0.9980
Epoch 6/50
275/275 [==============================] - 18s 67ms/step - loss: 0.1725 - acc: 0.9928 - val_loss: 0.1511 - val_acc: 0.9987
Epoch 7/50
275/275 [==============================] - 18s 67ms/step - loss: 0.1680 - acc: 0.9934 - val_loss: 0.1487 - val_acc: 0.9980
Epoch 8/50
275/275 [==============================] - 18s 67ms/step - los

In [20]:
# Evaluate the model
t1=time()
y_proba = model.predict(X_test)
t2=time()
print  ('Took {} sec to predict {} images'.format(t2-t1, len(X_test)) )

pred_labels = []
pred_probas = []
for i in range(0,len(y_proba)):
    classId = y_proba[i].argmax()
    pred_labels.append(classId)
    proba = max(y_proba[i])
    pred_probas.append(proba)

pred_labels = np.array(pred_labels, dtype='uint8')
pred_probas = np.array(pred_probas)

acc = np.mean(pred_labels==y_test)
print("Test accuracy = {}".format(acc*100))

Took 2.2971127033233643 sec to predict 12630 images
Test accuracy = 99.36658749010293


In [69]:
model.save('GTSRB/german_aug-99.37.h5')